In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [ ]:
!mkdir -p /content/yolov5/data/scripts
# COCO2YOLO.py
!wget -O /content/yolov5/data/scripts/COCO2YOLO.py https://raw.githubusercontent.com/alexmihalyk23/COCO2YOLO/3d04bf84e6252166d04ed465c58ee481a421e68e/COCO2YOLO.py

--2024-05-19 06:51:00--  https://raw.githubusercontent.com/alexmihalyk23/COCO2YOLO/3d04bf84e6252166d04ed465c58ee481a421e68e/COCO2YOLO.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4299 (4.2K) [text/plain]
Saving to: ‘/content/yolov5/data/scripts/COCO2YOLO.py’

/content/yolov5/dat 100%[===================>]   4.20K  --.-KB/s    in 0s      

2024-05-19 06:51:00 (62.7 MB/s) - ‘/content/yolov5/data/scripts/COCO2YOLO.py’ saved [4299/4299]



In [ ]:
# COCO 데이터셋
!curl -L "http://images.cocodataset.org/zips/train2017.zip" > train2017.zip
!curl -L "http://images.cocodataset.org/zips/val2017.zip" > val2017.zip
!curl -L "http://images.cocodataset.org/annotations/annotations_trainval2017.zip" > annotations_trainval2017.zip

!unzip -q train2017.zip -d datasets/
!unzip -q val2017.zip -d datasets/
!unzip -q annotations_trainval2017.zip -d datasets/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.0G  100 18.0G    0     0  48.8M      0  0:06:17  0:06:17 --:--:-- 71.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  777M  100  777M    0     0  53.6M      0  0:00:14  0:00:14 --:--:-- 63.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  241M  100  241M    0     0  91.0M      0  0:00:02  0:00:02 --:--:-- 91.0M


In [ ]:
import json
from pathlib import Path
import os

train_annotations_path = 'datasets/annotations/instances_train2017.json'
val_annotations_path = 'datasets/annotations/instances_val2017.json'
train_images_dir = 'datasets/train2017'
val_images_dir = 'datasets/val2017'

vehicle_class_ids = [2, 3, 4, 6, 8]

def filter_coco_annotations(input_path, output_path, class_ids):
    with open(input_path) as f:
        data = json.load(f)

    filtered_images = []
    filtered_annotations = []
    image_ids = set()

    for annotation in data['annotations']:
        if annotation['category_id'] in class_ids:
            filtered_annotations.append(annotation)
            image_ids.add(annotation['image_id'])

    for image in data['images']:
        if image['id'] in image_ids:
            filtered_images.append(image)

    filtered_data = {
        'images': filtered_images,
        'annotations': filtered_annotations,
        'categories': [cat for cat in data['categories'] if cat['id'] in class_ids]
    }

    with open(output_path, 'w') as f:
        json.dump(filtered_data, f)

    return image_ids

def delete_unnecessary_images(image_dir, keep_image_ids):
    for image_file in os.listdir(image_dir):
        image_id = int(Path(image_file).stem.lstrip('0'))
        if image_id not in keep_image_ids:
            os.remove(os.path.join(image_dir, image_file))

train_keep_ids = filter_coco_annotations(train_annotations_path, 'datasets/annotations/filtered_train2017.json', vehicle_class_ids)
val_keep_ids = filter_coco_annotations(val_annotations_path, 'datasets/annotations/filtered_val2017.json', vehicle_class_ids)

delete_unnecessary_images(train_images_dir, train_keep_ids)
delete_unnecessary_images(val_images_dir, val_keep_ids)


차량 관련 이미지만 남기고 필터링 및 삭제가 완료되었습니다.


In [ ]:
!python data/scripts/COCO2YOLO.py -j datasets/annotations/filtered_train2017.json -o datasets/yolo_format/train
!python data/scripts/COCO2YOLO.py -j datasets/annotations/filtered_val2017.json -o datasets/yolo_format/val

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
126590 [('000000126590.jpg', 3, (0.7728541666666666, 0.1555859375, 0.08291666666666667, 0.045140625000000004)), ('000000126590.jpg', 3, (0.7194895833333333, 0.14686718750000002, 0.041312499999999995, 0.032421875)), ('000000126590.jpg', 3, (0.1391770833333333, 0.13012500000000002, 0.18643749999999998, 0.0489375)), ('000000126590.jpg', 3, (0.5330520833333333, 0.1308828125, 0.019270833333333334, 0.016859375)), ('000000126590.jpg', 3, (0.9913645833333332, 0.1459296875, 0.017270833333333332, 0.046921875))]
409771 [('000000409771.jpg', 3, (0.5870078125, 0.5036875000000001, 0.063859375, 0.03216666666666666)), ('000000409771.jpg', 3, (0.7132499999999999, 0.51809375, 0.04403125, 0.0398125)), ('000000409771.jpg', 3, (0.405484375, 0.5194479166666667, 0.048156250000000005, 0.0280625)), ('000000409771.jpg', 3, (0.7477968750000001, 0.56421875, 0.06478125, 0.02372916666666667)), ('000000409771.jpg', 3, (0.5332187500000001, 0.5055729166666667, 0.02490625, 0.01872916

In [ ]:
!mv /content/yolov5/datasets/yolo_format/val/*.txt /content/yolov5/datasets/val2017/
!mv /content/yolov5/datasets/yolo_format/train/*.txt /content/yolov5/datasets/train2017/

In [ ]:
%%writefile data/vehicle.yaml
train: datasets/train2017
val: datasets/val2017

nc: 4  # 클래스 수
names: ['bicycle', 'car', 'motorcycle', 'bus', 'truck']  # 클래스 이름

Overwriting data/vehicle.yaml


In [ ]:
!python train.py --img 640 --batch 16 --epochs 100 --data data/vehicle.yaml --weights yolov5s.pt

python3: can't open file '/content/train.py': [Errno 2] No such file or directory
